In [1]:
from pandasticsearch import Select
from elasticsearch import Elasticsearch
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()

In [2]:
#엘라스틱서치 연결
es = Elasticsearch(hosts='192.168.1.66', port=9206, timeout=30, max_retries=10, retry_on_timeout=True)

In [10]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size,
        "sort": [
  {
    "send_timestamp": {
      "order": "desc"
    }
  }
], 
        "query": {
    "bool":{
      "filter": [
        {
          "exists": {
            "field": "NamedEntity"
          }
        }, 
        {
          "range": {
            "send_timestamp": {
              "gte": "2021-08-11T00:00:00.000Z",
              "format": "strict_date_optional_time"
            }
          }
        }
        
        ,{"terms": {
          "YNewsML.Metadata.Class.ClassCode.code": [
            "606001",
            "606002",
            "606003",
            "606004",
            "606005",
            "606006",
            "606007",
            "606008",
            "606009",
            "606010",
            "606011",
            "606012",
            "606013",
            "607001",
            "607002",
            "607003",
            "607004",
            "607005",
            "607006"
          ]
        }}
        
      ]
  }}}
    return body

In [11]:
index = 'rens_test'
size = 1000
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

total

13474

In [12]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

100%|██████████| 3/3 [00:04<00:00,  1.49s/it]


In [13]:
df

,_index,_type,_id,_score,YNewsML.Header.SendTime,YNewsML.Header.Action,YNewsML.Header.ServiceType,YNewsML.Header.ContentID,YNewsML.Header.SendDate,YNewsML.Metadata.Category.code,...,YNewsML.Metadata.Attribute.AttributeCode.code,YNewsML.Metadata.Attribute.AttributeCode.name,YNewsML.Metadata.Class.ClassCode.code,YNewsML.Metadata.Class.ClassCode.name,YNewsML.RefContent.Title,YNewsML.RefContent.Href,YNewsML.RefContent.ContentID,YNewsML.RefContent.type,YNewsML.Metadata.Attribute.AttributeCode,YNewsML.Metadata.Embargo
0,rens_test,_doc,AKR20221231041400061,None,232059,C,AKR0,AKR20221231041400061,20221231,07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rens_test,_doc,AKR20221231035000099,None,180813,C,AKR0,AKR20221231035000099,20221231,06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rens_test,_doc,AKR20221231025151003,None,161620,U,AKR0,AKR20221231025151003,20221231,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rens_test,_doc,AKR20221231025100003,None,125734,C,AKR0,AKR20221231025100003,20221231,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rens_test,_doc,AKR20221231022000076,None,115125,C,AKR0,AKR20221231022000076,20221231,06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3469,rens_test,_doc,AKR20220608057300003,None,103309,C,AKR0,AKR20220608057300003,20220608,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3470,rens_test,_doc,AKR20220607103300530,None,100007,C,AKR0,AKR20220607103300530,20220608,03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3471,rens_test,_doc,PYH20220608015200017,None,092049,C,PYH0,PYH20220608015200017,20220608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3472,rens_test,_doc,AKR20220607168000017,None,090002,C,AKR0,AKR20220607168000017,20220608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df1 = df.reset_index().drop("index",axis=1)

In [23]:
df1

,_index,_type,_id,_score,YNewsML.Header.SendTime,YNewsML.Header.Action,YNewsML.Header.ServiceType,YNewsML.Header.ContentID,YNewsML.Header.SendDate,YNewsML.Metadata.Category.code,...,YNewsML.Metadata.Attribute.AttributeCode.code,YNewsML.Metadata.Attribute.AttributeCode.name,YNewsML.Metadata.Class.ClassCode.code,YNewsML.Metadata.Class.ClassCode.name,YNewsML.RefContent.Title,YNewsML.RefContent.Href,YNewsML.RefContent.ContentID,YNewsML.RefContent.type,YNewsML.Metadata.Attribute.AttributeCode,YNewsML.Metadata.Embargo
0,rens_test,_doc,AKR20221231041400061,None,232059,C,AKR0,AKR20221231041400061,20221231,07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rens_test,_doc,AKR20221231035000099,None,180813,C,AKR0,AKR20221231035000099,20221231,06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rens_test,_doc,AKR20221231025151003,None,161620,U,AKR0,AKR20221231025151003,20221231,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rens_test,_doc,AKR20221231025100003,None,125734,C,AKR0,AKR20221231025100003,20221231,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rens_test,_doc,AKR20221231022000076,None,115125,C,AKR0,AKR20221231022000076,20221231,06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13469,rens_test,_doc,AKR20220608057300003,None,103309,C,AKR0,AKR20220608057300003,20220608,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13470,rens_test,_doc,AKR20220607103300530,None,100007,C,AKR0,AKR20220607103300530,20220608,03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13471,rens_test,_doc,PYH20220608015200017,None,092049,C,PYH0,PYH20220608015200017,20220608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13472,rens_test,_doc,AKR20220607168000017,None,090002,C,AKR0,AKR20220607168000017,20220608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df1.columns.tolist()

['_index',
 '_type',
 '_id',
 '_score',
 'YNewsML.Header.SendTime',
 'YNewsML.Header.Action',
 'YNewsML.Header.ServiceType',
 'YNewsML.Header.ContentID',
 'YNewsML.Header.SendDate',
 'YNewsML.Metadata.Category.code',
 'YNewsML.Metadata.Category.name',
 'YNewsML.Metadata.Credit',
 'YNewsML.Metadata.DeliCode',
 'YNewsML.Metadata.Region.RegionCode',
 'YNewsML.Metadata.Href',
 'YNewsML.Metadata.Desk.code',
 'YNewsML.Metadata.Desk.content',
 'YNewsML.Metadata.Class.ClassCode',
 'YNewsML.Metadata.Urgency',
 'YNewsML.Metadata.Source',
 'YNewsML.Metadata.Writer.uid',
 'YNewsML.Metadata.Writer.content',
 'YNewsML.NewsContent.LangType',
 'YNewsML.NewsContent.Title',
 'YNewsML.NewsContent.Body',
 'knn_vector',
 'NamedEntity',
 'Keyword',
 'send_timestamp',
 'Summarization',
 'Quotation',
 'YNewsML.Metadata.Region.RegionCode.code',
 'YNewsML.Metadata.Region.RegionCode.name',
 'YNewsML.NewsContent.Comment',
 'YNewsML.NewsContent.SubTitle',
 'YNewsML.RefContent',
 'YNewsML.Metadata.Category',
 'YNew

In [27]:
df2 = df1.drop_duplicates(subset = 'YNewsML.NewsContent.Body')

In [28]:
df2

,_index,_type,_id,_score,YNewsML.Header.SendTime,YNewsML.Header.Action,YNewsML.Header.ServiceType,YNewsML.Header.ContentID,YNewsML.Header.SendDate,YNewsML.Metadata.Category.code,...,YNewsML.Metadata.Attribute.AttributeCode.code,YNewsML.Metadata.Attribute.AttributeCode.name,YNewsML.Metadata.Class.ClassCode.code,YNewsML.Metadata.Class.ClassCode.name,YNewsML.RefContent.Title,YNewsML.RefContent.Href,YNewsML.RefContent.ContentID,YNewsML.RefContent.type,YNewsML.Metadata.Attribute.AttributeCode,YNewsML.Metadata.Embargo
0,rens_test,_doc,AKR20221231041400061,None,232059,C,AKR0,AKR20221231041400061,20221231,07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rens_test,_doc,AKR20221231035000099,None,180813,C,AKR0,AKR20221231035000099,20221231,06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rens_test,_doc,AKR20221231025151003,None,161620,U,AKR0,AKR20221231025151003,20221231,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rens_test,_doc,AKR20221231025100003,None,125734,C,AKR0,AKR20221231025100003,20221231,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rens_test,_doc,AKR20221231022000076,None,115125,C,AKR0,AKR20221231022000076,20221231,06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13469,rens_test,_doc,AKR20220608057300003,None,103309,C,AKR0,AKR20220608057300003,20220608,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13470,rens_test,_doc,AKR20220607103300530,None,100007,C,AKR0,AKR20220607103300530,20220608,03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13471,rens_test,_doc,PYH20220608015200017,None,092049,C,PYH0,PYH20220608015200017,20220608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13472,rens_test,_doc,AKR20220607168000017,None,090002,C,AKR0,AKR20220607168000017,20220608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df2.to_csv('Final_fire_news.csv', encoding = 'utf-8')